In [0]:
def save_table(df, table_name, layer):
    (
        df.write.mode("append").format('delta')
        .option("mergeSchema", "true")    
        .option("delta.columnMapping.mode", "name")   
        .partitionBy("ANO_DO_EXERCICIO", "BAIRRO") 
        .saveAsTable(f"{layer}_layer.{table_name}")
    )

In [0]:
def ingestao_dados(dados, table_name):

    # Leitura dos dados da tabela existente (caso já tinha sido salva anteriormente)
    dados_existentes = spark.read.table(f"gold_layer.{table_name}")
    
    # Verificando se os dados importados já existem
    verifica_tabela = spark.catalog.tableExists(f"gold_layer.{table_name}")

    # Se a tabela já existir, realiza filtragem no dataframe importado para inserir apenas os dados novos
    if verifica_tabela:         

        colunas = dados.columns

        condicoes = [dados[c] == dados_existentes[c] for c in colunas]

        df_filtrado = dados.join(dados_existentes, on=condicoes, how="left_anti")
        
        df_filtrado = df_filtrado.dropDuplicates()

        print('================================== CARGA DE DADOS ==================================')
        print('>> Camada: Gold')
        print(f'>> Caminho da tabela silver: silver_layer.{table_name}')
        print('>>> Executando carga de dados, aguarde...')

        try:        
            save_table(df_filtrado, table_name, 'gold')
            print('=========================== CARGA FINALIZADA COM SUCESSO ===========================\n')
        except:
            print('============================= ERRO NA CARGA DE DADOS... ============================\n')    

    # Caso a tabela não exista, realiza a carga dos dados pela primeira vez
    else:        
        
        print('================================== CARGA DE DADOS ==================================')
        print('>> Camada: Gold')
        print(f'>> Caminho da tabela silver: silver_layer.{table_name}')
        print('>>> Executando carga de dados, aguarde...')

        dados = dados.dropDuplicates()

        try:        
            save_table(dados, table_name, 'gold')
            print('=========================== CARGA FINALIZADA COM SUCESSO ===========================\n')
        except:
            print('============================= ERRO NA CARGA DE DADOS... ============================\n')